<a href="https://colab.research.google.com/github/sohomghosh/FiNCAT_Financial_Numeral_Claim_Analysis_Tool/blob/main/tool/FiNCAT_tool_enhanced_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#@title

%%capture
!wget https://raw.githubusercontent.com/sohomghosh/FiNCAT_Financial_Numeral_Claim_Analysis_Tool/main/fincat_utils.py
!wget https://raw.githubusercontent.com/sohomghosh/FiNCAT_Financial_Numeral_Claim_Analysis_Tool/main/tool/lr_clf_FiNCAT.pickle
!pip install transformers
!pip install gradio
import gradio as gr
import nltk
import pandas as pd
nltk.download('punkt')
from fincat_utils import extract_context_words
from fincat_utils import bert_embedding_extract
import pickle
lr_clf = pickle.load(open("lr_clf_FiNCAT.pickle",'rb'))

In [7]:
#@title
#txt = 'In the year 2021 the markets were bullish. We expect to boost our sales by 80% this quarter.'
def score_fincat(txt):
  li = []
  highlight = []
  for word in txt.split():
    if any(char.isdigit() for char in word):
      if word[-1] in ['.', ',', ';', ":", "-", "!", "?", ")", '"', "'"]:
        word = word[:-1]
      st = txt.index(word)
      ed = st + len(word)
      x = {'paragraph' : txt, 'offset_start':st, 'offset_end':ed}
      context_text = extract_context_words(x)
      features = bert_embedding_extract(context_text, word)
      prediction = lr_clf.predict(features.reshape(1, 768))
      prediction_probability = '{:.4f}'.format(round(lr_clf.predict_proba(features.reshape(1, 768))[:,1][0], 4))
      highlight.append((word, '    In-claim' if prediction==1 else 'Out-of-Claim'))
      li.append([word,'    In-claim' if prediction==1 else 'Out-of-Claim', prediction_probability])
    else:
      highlight.append((word, '    '))
  headers = ['numeral', 'prediction', 'probability']
  dff = pd.DataFrame(li)
  dff.columns = headers
  
  return highlight, dff

iface = gr.Interface(fn=score_fincat, inputs=gr.inputs.Textbox(lines=5, placeholder="Enter Financial Text here..."), title="FiNCAT-2",description="Financial Numeral Claim Analysis Tool (Enhanced)", outputs=["highlight", "dataframe"], allow_flagging="never", examples=["In the year 2021, the markets were bullish. We expect to boost our sales by 80% this year.", "Last year our profit was $2.2M. This year it will increase to $3M"])
iface.launch()





Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://17160.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f443d673310>,
 'http://127.0.0.1:7860/',
 'https://17160.gradio.app')